In [ ]:
import base64
from datetime import (
    datetime,
)
from enum import (
    Enum,
)
from flask import (
    abort,
    Flask,
    jsonify,
    request,
    Response,
)
from flask_cors import (
    CORS,
)
import io
import json
import matplotlib.pyplot as plt
import networkx as nx
import requests
import sys
import time
from wordcloud import (
    WordCloud,
)

def post_json_request(url, obj):
    return requests.post(url, json=obj).json()

def object_to_response(object):
    response = Response(
        response=json.dumps(object), mimetype="application/json"
    )
    response.headers["Access-Control-Allow-Origin"] = "*"

    return response

def adjacency_pipeline(url, organization, project, pattern, graph_type):
    success = 0
    nodes = 0
    edges = 0
    b64_image = "data:image/png;base64,"
    wordcloud_b64_image = "data:image/png;base64,"
    node_link_data = {}
    try:
        patterns: list[dict[str, str]] = post_json_request(
            "http://db:5000/mongo-doc-list",
            {"db_name": organization, "coll_name": f"patterns#{project}"},
        )
        singular = graph_type[:-1]
        if pattern == "global" or (
            patterns and pattern in [item["_id"] for item in patterns]
        ):
            items = post_json_request(
                "http://db:5000/mongo-doc-list",
                {
                    "db_name": organization,
                    "coll_name": f"{graph_type}#{project}#{pattern}",
                },
            )
            items_list = [
                f'{item[singular]} {" ".join(item["related"])}'
                for item in items
            ]
            G = nx.parse_adjlist(items_list, nodetype=str)
            plt.figure(figsize=(30, 30))
            nx.draw(
                G,
                with_labels=True,
                node_color="skyblue",
                node_size=100,
                font_size=10,
                style="solid",
            )
            nodes = G.number_of_nodes()
            edges = G.number_of_edges()
            image_bytes = io.BytesIO()
            plt.savefig(image_bytes, format="jpg")
            image_bytes.seek(0)
            b64_image = (
                f"{b64_image}{base64.b64encode(image_bytes.read()).decode()}"
            )
            node_link_data = nx.node_link_data(G)

            keyword_items = post_json_request(
                "http://db:5000/mongo-doc-list",
                {
                    "db_name": organization,
                    "coll_name": f"wordcloud#{project}#{pattern}",
                },
            )
            keyword_list = [item["keyword"] for item in keyword_items]
            text = " ".join(keyword_list)
            # mask = np.array(Image.open(path.join(d, "batman.jpg")))
            wc = WordCloud(
                background_color="black",
                max_words=3000,
                # mask=mask,
                max_font_size=30,
                min_font_size=0.1,
                random_state=42,
            )
            wc.generate(text)
            plt.figure(figsize=(30, 30))
            plt.imshow(wc, interpolation="bilinear")
            plt.axis("off")
            wordcloud_image_bytes = io.BytesIO()
            plt.savefig(
                wordcloud_image_bytes,
                bbox_inches="tight",
                format="jpg",
            )
            wordcloud_image_bytes.seek(0)
            wordcloud_b64_image = f"{wordcloud_b64_image}{base64.b64encode(wordcloud_image_bytes.read()).decode()}"

    except Exception as ex:
        print(f"Error: {ex}")
        success = 1
    return {
                "exit": success,
                "nodes": nodes,
                "edges": edges,
                "b64_image": b64_image,
                "node_link_data": node_link_data,
                "wordcloud_image": wordcloud_b64_image
                if wordcloud_b64_image is not None
                else "",
            }